In [1]:
%load_ext autoreload
%autoreload 2
from build123d import *
from jupyter_cadquery import set_defaults, show, Camera, Collapse
import math
from functools import reduce, partial
from types import SimpleNamespace
from typing import TypeVar, Optional
from dataclasses import dataclass
from ipywidgets import interact
import ipywidgets as widgets
from pathlib import Path

set_defaults(
    viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

from projects.lockbox import Dividers, FixedDividerLockboxLid, FixedDividerLockboxBody
import inspect

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [63]:
from pathlib import Path

#print(Path("../../../packages/projects/src/projects/lockbox/__init__.py").read_text())

In [64]:
length = 250
width = 100
height = 32
thickness = 4.2
lid_height = 1.2
margin = 0.21
body_curvature = 2
divider_thickness = 1.26
num_dividers = 11
lip_chamfer = 1

outer_length, outer_width = length, width
inner_length, inner_width = length - 2 * thickness, width - 2 * thickness
lip_length, lip_width = length - thickness - margin / 2, width - thickness - margin / 2
inner_height = height - 2 * thickness
lip_height = height - thickness

with BuildPart() as body:
    Box(outer_length, outer_width, height, align=(Align.CENTER, Align.CENTER, Align.MIN))
    with Locations((0,0, height)):
        Box(lip_length, lip_width, thickness + margin, align=(Align.CENTER, Align.CENTER, Align.MAX), mode=Mode.SUBTRACT)
    with Locations((0,0,height - thickness)):
        Box(inner_length, inner_width, inner_height,align=(Align.CENTER, Align.CENTER, Align.MAX), mode=Mode.SUBTRACT)
    fillet(body.edges().filter_by(Axis.Z), body_curvature)
    with BuildSketch(body.faces().filter_by(Axis.Z).last) as divider_profile:
        with GridLocations(inner_length / (num_dividers + 1), 1, num_dividers, 1):
            Rectangle(divider_thickness, inner_width + thickness / 2, align=(Align.CENTER, Align.CENTER))
    with BuildPart(body.faces().filter_by(Axis.Z).sort_by(Axis.Z)[-2]) as dividers:
        add(divider_profile)
        extrude(amount=-(inner_height + thickness / 2))
    add(dividers, mode=Mode.SUBTRACT)
    chamfer_faces = body.faces().sort_by(Axis.Z).filter_by(Axis.Z)
    chamfer_edges = chamfer_faces[0].edges()
    chamfer(chamfer_edges, thickness / 4, thickness / 4)

with BuildPart() as lid:
    Box(outer_length, outer_width, thickness, align=(Align.CENTER, Align.CENTER, Align.MIN))
    Box(inner_length, inner_width, thickness, align=(Align.CENTER, Align.CENTER, Align.MAX))
    fillet(lid.edges().filter_by(Axis.Z), body_curvature)
    chamfer(lid.faces().filter_by(Axis.Z).edges(), thickness / 4, thickness / 4)

show(Plane.XY.offset(height) * lid.part, body)

cc


In [10]:
DIVIDER_MASK = [False] * 5 + [True] + [False] * 6

body = FixedDividerLockboxBody(
    length = 240,
    width = 100,
    height = 30,
    thickness = 1.26,
    body_roundness = 8,
    divider_mask = DIVIDER_MASK
)
lid = FixedDividerLockboxLid(
    length = 240,
    width = 100,
    height = 30,
    lid_height = 1.6,
    thickness = 1.26,
    body_roundness = 8,
    margin = 0.21,
    divider_mask = DIVIDER_MASK
)

assembly = Compound(label="Assembly", children=[Plane.XY.offset(30 + 1.6 + 1.26) * lid, body])
show(assembly)

[(0.0, 0)]
++


In [62]:
inside = Pos(0,0,HEIGHT / 2 + THICKNESS / 2) * Box(LENGTH, WIDTH, HEIGHT)

def __is_outer_edge(edge: Edge, y_bound: float, x_bound: float) -> bool:
    x, y, _ = edge.center().to_tuple()
    return (abs(y) == y_bound or abs(x) == x_bound)

def __is_inner_body_edge(edge: Edge, y_bound: float, x_bound: float, thickness: float) -> bool:
    x, y, z = edge.center().to_tuple()
    return -x_bound < x < x_bound and -y_bound <= y <= y_bound and round(z, 2) >= thickness 

def make_body(
        length: float,
        width: float,
        height: float,
        thickness: float,
        margin: float,
        lid_height: float,
        body_fillet: float,
        num_dividers: int,
        divider_thickness: float,
        chamfer_length: float,
        outer_bound: bool = True
    ):
    
    if outer_bound:
        outer_length, outer_width = length, width
        inner_length, inner_width = length - 2 * thickness, width - 2 * thickness
        lip_length, lip_width = length - thickness - margin / 2, width - thickness - margin / 2
        inner_height = height - 2 * thickness
        lip_height = height - thickness
    else:
        inner_length, inner_width = length, width
        outer_length, outer_width = length + 2 * thickness, width + 2 * thickness
        lip_length, lip_width = length + thickness - margin / 2, width + thickness - margin / 2
        inner_height = height
        lip_height = height + thickness
    
    base_profile = Rectangle(outer_length, outer_width)
    lip_bottom_profile = base_profile - Rectangle(inner_length, inner_width)
    lip_top_profile = base_profile - Rectangle(lip_length, lip_width)
    divider_profile = Rectangle(divider_thickness, inner_width + thickness / 2)

    obj = extrude(
        base_profile, thickness
    ) + extrude(
        Plane.XY.offset(thickness) * lip_bottom_profile, inner_height
    )
    
    if lid_height < thickness:
        obj = obj + extrude(
            Plane.XY.offset(lip_height) * lip_top_profile, lid_height
        )
    elif lid_height > thickness:
        obj = obj - extrude(
            Plane.XY.offset(lip_height).reverse() * lip_top_profile, lid_height
        )
    
    if body_fillet > 0:
        obj = fillet(obj.edges().filter_by(Axis.Z), body_fillet)
    
    if num_dividers > 0:
        divider_cutout = extrude(Plane.XY.offset(lip_height) * divider_profile, -(inner_height + thickness / 2))

        obj: Part = obj - reduce(lambda x, y: x + y, [
            loc * divider_cutout for loc in GridLocations(inner_length / (num_dividers + 1), 1, num_dividers, 1)
        ])

    if chamfer_length > 0:
        should_chamfer = partial(__is_outer_edge, x_bound=outer_length / 2, y_bound=outer_width / 2)
        obj = chamfer(ShapeList(filter(should_chamfer, obj.edges())), chamfer_length, chamfer_length)
        if num_dividers > 0:
            should_fillet = partial(__is_inner_body_edge, x_bound=inner_length / 2, y_bound=inner_width / 2, thickness=thickness)
            obj = fillet(ShapeList(filter(should_fillet, obj.edges().filter_by(Plane.YZ))), chamfer_length)
    
    return obj

def make_lid(
        length: float,
        width: float,
        thickness: float,
        margin: float,
        lid_height: float,
        body_fillet: float,
        chamfer_length: float,
        outer_bound: bool =True
    ):

    # Dimension correction

    if outer_bound:
        outer_length, outer_width = length, width
        lid_length, lid_width = length - thickness + margin / 2, width - thickness + margin / 2
    else:
        outer_length, outer_width = length + 2 * thickness, width + 2 * thickness
        lid_length, lid_width = length + thickness + margin / 2, width + thickness + margin / 2

    # Basic Shape

    base_profile = Rectangle(outer_length, outer_width)
    lid_top_profile = base_profile - Rectangle(lid_length, lid_width)
    obj = extrude(
        base_profile, thickness
    )

    # Lid correction

    if lid_height < thickness:
        obj = obj - extrude(
            Plane.XY.offset(thickness).reverse() * lid_top_profile, lid_height
        )
    elif lid_height > thickness:
        obj = obj + extrude(
            Plane.XY.offset(thickness) * lid_top_profile, lid_height
        )

    # Fillet body

    if body_fillet > 0:
        obj = fillet(obj.edges().filter_by(Axis.Z), body_fillet)

    # Chamfer edges

    if chamfer_length > 0:
        should_chamfer = partial(__is_outer_edge, x_bound=outer_length / 2, y_bound=outer_width / 2)
        obj = chamfer(ShapeList(filter(should_chamfer, obj.edges())), chamfer_length, chamfer_length)

    return obj

def make_divider(divider_thickness, inner_width, inner_height, thickness):
    return extrude(Rectangle(divider_thickness, inner_width), inner_height - thickness)

def assemble(
        length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation=True
    ):
    lid_pos = Pos((0,0,height)) * Rot((0,180,0)) if not printable_orientation else Pos((0, width + margin))
    divider_pos = Pos((0,0,height - thickness)) * Rot((0,180,0)) if not printable_orientation else Pos((0, -width, divider_thickness / 2)) * Rot((0,90,0))
    return (
        make_body(
            length, width, height, thickness, margin, lid_height, body_fillet, num_dividers, divider_thickness, chamfer_length
        ),
        lid_pos * make_lid(length, width, thickness, margin, lid_height, body_fillet, chamfer_length),
        divider_pos * make_divider(divider_thickness, width - 2 * thickness, height - thickness, thickness)
    )

def show_assembly(length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation):
    show(
        *assemble(length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation),
        #*assemble(226, 86, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation),
        body, lid,
        names=["body", "lid", "divider"] + ["body", "lid", "divider"],
        ticks=30
    )

show_assembly(
    LENGTH,
    WIDTH,
    HEIGHT,
    THICKNESS,
    MARGIN, LID_HEIGHT, BODY_FILLET, DIVIDER_THICKNESS, NUM_DIVIDERS, LIP_CHAMFER, False)

Too many names, trimming to length 5
+++c+


In [ ]:
length = 240
width = 100
height = 20
lid_height = 1.5
thickness = 1
body_fillet = 5
margin = 0.2
num_compartments = 12
#divider_mask = [False] * num_compartments
#divider_mask = [True] * num_compartments
divider_mask = [False, False, True, False, False, True, False, False, False, False, False]

In [61]:
LENGTH = 250
WIDTH = 100
HEIGHT = 32
THICKNESS = 4.2
LID_HEIGHT = 1.2
MARGIN = 0.21
BODY_FILLET = 2
DIVIDER_THICKNESS = 1.26
NUM_DIVIDERS = 11
LIP_CHAMFER = 1

In [ ]:
body, lid, divider = assemble(
    LENGTH,
    WIDTH,
    HEIGHT,
    THICKNESS,
    MARGIN,
    LID_HEIGHT,
    BODY_FILLET,
    DIVIDER_THICKNESS,
    NUM_DIVIDERS,
    LIP_CHAMFER,
)
show(body, lid, divider)

In [ ]:
from utilities.export import export
from pathlib import Path

body, lid, divider = assemble(60, 60, 30, THICKNESS, MARGIN, LID_HEIGHT, BODY_FILLET, DIVIDER_THICKNESS, NUM_DIVIDERS, LIP_CHAMFER)

export(Path("outputs") / "Lockbox" / "60mmX60mmX30mm", body, lid, divider)

In [ ]:
output_path : Path = Path("outputs") / "Lockbox" / f"{LENGTH}mmx{WIDTH}mmx{HEIGHT}mm"
output_path.mkdir(exist_ok=True)
export_stl(body, output_path / "body.stl")
export_stl(lid, output_path / "lid.stl")
export_stl(divider, output_path / "divider.stl")